# FRRouting + OSPF Slice Builder

For building a single-site FABRIC Slice that utilizes the FRRouting implementation of OSPFv2.

The official FRRouting documentation for OSPFv2 can be found here: https://docs.frrouting.org/en/stable-5.0/ospfd.html

The following OSPF RFC's are implemented:
* RFC 2328 OSPF Version 2. J. Moy. April 1998.
* RFC 2370 The OSPF Opaque LSA Option R. Coltun. July 1998.
* RFC 3101 The OSPF Not-So-Stubby Area (NSSA) Option P. Murphy. January 2003.
* RFC 3137 OSPF Stub Router Advertisement, A. Retana, L. Nguyen, R. White, A. Zinin, D. McPherson. June 2001

RFC 2740, OSPv3 (for IPv6) is also implemented by FRR, but is currently not considered in this book.

This book is split into three phases that are defined by four steps:

1. Input data about the topology (such as the site to use, if there are client nodes, etc).

2. Input the topology itself so that nodes and their interconnections are defined. The topology is parsed via a GraphML file.

3. Create a Slice based on the data collected in steps 1 and 2 and submit it to FABRIC for creation.

4. Execute FRRouting configuration commands to build the OSPF topology.

<div>
<img src="../figs/ospf_fabric_diagram.png" width="800"/>
</div>

At this time, the Slice supernet is 192.168.0.0/16, but could be extended later to allow for a customizable IPv4 address space.

## Topology Configuration

| Variable | Use |
| --- | --- |
| SLICE_NAME    | Name of slice you want to create. Please make sure a slice with that name does not already exist. |
| SITE_NAME     | Name of the FABRIC site you want the nodes to be reserved at. This code does not consider inter-site situations, the entire topology is reserved on a single slice. |
| GRAPH_PATH    | Path to the graphml file you want to use to create a topology. |
| HAS_CLIENTS   | Enter True if clients are present in topology, if not, False. These nodes and the networks connecting them utilizes alternative naming and addressing structures. |
| CLIENT_PREFIX | The naming prefix given to each node (currently, this is required if the topology does have clients). |
| MEAS_ADD      | Enter True if measurements are to be taken on the slice. This requires the inclusion of a separate measurement node. |

In [1]:
SLICE_NAME = "ospf_no_meas"
SITE_NAME = "WASH"
GRAPH_PATH = "/home/fabric/work/FABRIC-EIBP/graphs/ospf_small.graphml"
HAS_CLIENTS = True
CLIENT_PREFIX = "ipnode"
#MEAS_ADD = True

## Import the FABlib Library and Confirm the Configuration is Correct

If you receive an exception when executing this cell, something about your FABRIC configuration is incorrect or inputted wrong. Changes to this book will not fix this issue, so you will have to find the reason for the failure (or ask for help).

In [9]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try: 
    fablib = fablib_manager()
                     
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,787adfc9-d37e-42f2-8efe-8e32793e0bb8
Bastion Username,kg8849_0000103312
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


## Parse the GraphML for the Topology and Create the Slice

The minidom library is used to parse the graphml. It assumes proper use of the node and edge tags. Examples of valid tags can be seen below. The name of the node is the node's id. Furthermore, it does not matter which node is the source and destination, it is parsed as an undirected graph and there will be issues if you try and create another network between the same two nodes.

    <node id="L1" />
    <node id="S1" />
    <edge source="L1" target="S1" />
    
<b>NOTE:</b> FRR is installed, followed by OSPF configuration, via a post-boot script found in the remote_scripts directory. It makes assumptions on the IPv4 address space that is utilized based on the default configuration in this notebook. Any changes to IP addressing will require the script to be updated as well. Making this process dynamic can be accomplished in an update later.

In [3]:
import xml.dom.minidom
from collections import Counter

STARTING_INTF_NUM = 1

try:        
    #Create the slice
    slice = fablib.new_slice(name=SLICE_NAME)
    
    # Create dictionary to store nodes
    nodeDict = {}
    
     # Create dictionary to store information to dump into a file
    logFile = {"name": SLICE_NAME, "site": SITE_NAME}
    
    # Use XML parser to parse the GraphML file
    docs = xml.dom.minidom.parse(GRAPH_PATH)

    # Find all nodes via the node tag, add each to the slice with Rocky Linux as its base
    nodes = docs.getElementsByTagName("node")
    for node in nodes:
        # Grab the node name and determine if it is a client node based on its name prefix
        nodeName = node.getAttribute("id")
        isClient = True if HAS_CLIENTS and nodeName.startswith(CLIENT_PREFIX) else False # Check for compute/client nodes
        
         
        ### LOG FILE INFO
        logFile[nodeName] = {"isClient": isClient, "networks": {}}
        
        
        # Add node to the slice
        nodeInfo = slice.add_node(name=nodeName, cores=1, ram=4, image='default_rocky_8', site=SITE_NAME)
        
        # nodeDict = 0 -> FABRIC node object, 1 -> current intf number, 2 -> is a client/server (True) or not (False)
        nodeDict[nodeName] = {"nodeInfo": nodeInfo, "currentIntfNum": STARTING_INTF_NUM, "isClient": isClient}
        #[nodeInfo, 1, isClient]
        
        # FRR installation script, to be run when the router nodes are first brought up.
        if(not isClient):
            nodeInfo.add_post_boot_upload_directory('../remote_scripts/frr_scripts','.')
            nodeInfo.add_post_boot_execute('chmod +x frr_scripts/init_ospf.sh && ./frr_scripts/init_ospf.sh')
        
        print(f'Added node {nodeName}')
    
    # Find all edges via the edge tag, add each to the slice via an L2Bridge connecting the node interfaces
    edges = docs.getElementsByTagName("edge")
    for edge in edges:
        # grab nodes x and y in edge (x,y)
        source = edge.getAttribute("source")
        target = edge.getAttribute("target")
        
        # Create an interface name for each interface in the network
        sourceIntfName = "eth{num}".format(node=source, num=nodeDict[source]["currentIntfNum"])
        targetIntfName = "eth{num}".format(node=target, num=nodeDict[target]["currentIntfNum"])
        
        # Name a network based on if it is a user-facing LAN (edge) or P2P links in the core of the network (core)
        networkPrefix = "edge" if nodeDict[source]["isClient"]  or nodeDict[target]["isClient"] else "core"
        networkName = f'{networkPrefix}-{source}-{target}'
        
        # Add a NIC for each node that is a part of the edge
        sourceIntf = nodeDict[source]["nodeInfo"].add_component(model='NIC_Basic', name=sourceIntfName).get_interfaces()[0]
        targetIntf = nodeDict[target]["nodeInfo"].add_component(model='NIC_Basic', name=targetIntfName).get_interfaces()[0]
        
        nodeDict[source]["currentIntfNum"] += 1
        nodeDict[target]["currentIntfNum"] += 1

        # Add a L2 network between the interfaces
        slice.add_l2network(name=networkName, interfaces=[sourceIntf, targetIntf], type="L2Bridge")
        
         ### LOG FILE INFO
        logFile[source]["networks"][networkName] = {"neighbor": target}
        logFile[target]["networks"][networkName] = {"neighbor": source}
        
        print(f'Added edge {source}-{target}')

except Exception as e:
    print(f"Exception: {e}")

Added node node1
Added node node2
Added node node3
Added node ipnode-1
Added node ipnode-2
Added edge node1-node2
Added edge node1-node3
Added edge node2-node3
Added edge node2-ipnode-1
Added edge node3-ipnode-2


## Submit the Slice

Submit the Slice configuration to FABRIC for processing. Depending on the site chosen, the topology configuration inputted, and your current FABRIC configuration, this may fail. If so, troubleshoot as necessary and retry.

In [4]:
%%time
try:
    # Submit Slice Request
    print(f'Submitting the new slice, "{SLICE_NAME}"...')
    slice.submit()
    print(f'{SLICE_NAME} creation done.')

except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()


Retry: 15, Time: 605 sec


ID,ab3f666a-c3b9-4a4c-85d7-875a274846ae
Name,ospf_no_meas
Lease Expiration (UTC),2024-02-15 15:50:51 +0000
Lease Start (UTC),2024-02-14 15:50:52 +0000
Project ID,787adfc9-d37e-42f2-8efe-8e32793e0bb8
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
e0870e0f-0f9c-490a-a2e8-e9548637fae3,ipnode-1,1,4,10,default_rocky_8,qcow2,wash-w3.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fef0:6e21,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fef0:6e21,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
c4092836-1418-472b-86c0-1e2471f85f9b,ipnode-2,1,4,10,default_rocky_8,qcow2,wash-w3.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fe0f:aecd,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fe0f:aecd,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
a453fa40-84fd-4674-ae6c-3bec1362163d,node1,1,4,10,default_rocky_8,qcow2,wash-w3.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fecb:dd6d,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fecb:dd6d,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
e2fbadc3-9929-44c9-8141-f6bfe95a24a6,node2,1,4,10,default_rocky_8,qcow2,wash-w3.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fee7:f4da,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fee7:f4da,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
90d424dd-cfea-4ca0-a32d-669fb0c59b80,node3,1,4,10,default_rocky_8,qcow2,wash-w3.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:fea9:b8f3,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:fea9:b8f3,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
2ceb0aec-bf2c-4145-9900-f48597a4d1c9,core-node1-node2,L2,L2Bridge,WASH,None,None,Active,
261dd448-598d-4fda-bc23-3801e1f94be3,core-node1-node3,L2,L2Bridge,WASH,None,None,Active,
72fa3b66-8904-4a2b-9322-57972119fa7e,core-node2-node3,L2,L2Bridge,WASH,None,None,Active,
98852c93-002e-4613-bf08-e0b82499c96b,edge-node2-ipnode-1,L2,L2Bridge,WASH,None,None,Active,
5b718284-0073-4aae-9eab-7e431cfc2cef,edge-node3-ipnode-2,L2,L2Bridge,WASH,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
node1-eth1-p1,p1,node1,core-node1-node2,100,config,,52:9F:F0:04:C0:CA,eth1,eth1,None,4
node1-eth2-p1,p1,node1,core-node1-node3,100,config,,52:E2:49:A2:85:E6,eth2,eth2,None,4
node2-eth2-p1,p1,node2,core-node2-node3,100,config,,66:45:EC:A1:D2:13,eth3,eth3,None,4
node2-eth3-p1,p1,node2,edge-node2-ipnode-1,100,config,,5E:4A:87:F8:43:D3,eth1,eth1,None,4
node2-eth1-p1,p1,node2,core-node1-node2,100,config,,5E:70:C8:B1:56:36,eth2,eth2,None,4
node3-eth1-p1,p1,node3,core-node1-node3,100,config,,5A:AF:A8:81:A0:11,eth2,eth2,None,4
node3-eth2-p1,p1,node3,core-node2-node3,100,config,,56:B7:A0:B0:86:ED,eth1,eth1,None,4
node3-eth3-p1,p1,node3,edge-node3-ipnode-2,100,config,,5A:AF:C2:97:58:83,eth3,eth3,None,4
ipnode-1-eth1-p1,p1,ipnode-1,edge-node2-ipnode-1,100,config,,66:50:05:25:AA:76,eth1,eth1,None,4
ipnode-2-eth1-p1,p1,ipnode-2,edge-node3-ipnode-2,100,config,,6A:17:6C:BB:CC:7B,eth1,eth1,None,4



Time to print interfaces 618 seconds
ospf_no_meas creation done.
CPU times: user 1min 3s, sys: 1.11 s, total: 1min 4s
Wall time: 10min 19s


## Save the SSH Commands for the Nodes in the Topology

In [5]:
with open(f"{SLICE_NAME}_ssh_cmds.txt", "w") as sshFile:
    for node in nodeDict:
        sshFile.write(f"{node}:\n")
        sshFile.write(f"{nodeDict[node]['nodeInfo'].get_ssh_command()}\n")

## Add Basic IPv4 Addressing

In this Slice builder, 192.168.0.0/16 is the address space utilized on the FABRIC Slice. In other words, 192.168.0.0/16 is the Slice supernet and all networks created on the Slice are subnets of this supernet address space.

Each network is a /24 subnet of this network. Edge networks have client/compute devices with lower address (ex: .1) and networking nodes with higher addresses.

In [6]:
from ipaddress import ip_address, IPv4Address, IPv4Network

def updateMeasNetworkName(node, nodeName, intfName):
        if("meas" not in nodeName): 
            node.execute(command=f"sudo ip link set dev {intfName} down")
            node.execute(command=f"sudo ip link set dev {intfName} name meas")
            node.execute(command=f"sudo ip link set dev meas up")
            
            print(f"\t{nodeName} {intfName} renamed meas")
        else:
            print(f"\tMeasurement node not modified")

        return

# Start with a 1 in the third octet
thirdOctet = 1

# For each newtork in the slice
for network in slice.get_networks():
    # Grab all of the usable host addresses for a new network (new third octet).
    networkAddress = f'192.168.{thirdOctet}.0/24'
    currentIPNetwork = IPv4Network(networkAddress)
    hostIPList = list(currentIPNetwork)[1:-1] # 1:-1 = remove network and broadcast address
    getNewNetwork = False
    
    # Determine if the current network is an edge or core network
    networkName = network.get_name()
    
    isEdgeNetwork = True if networkName.startswith("edge") else False
    
    # For each interface in the network
    for intf in network.get_interfaces():        
        intfName = intf.get_physical_os_interface_name()
        node = intf.get_node()
        nodeName = node.get_name()
        
        # Measurement network is auto-assigned IP addressing, it needs intf naming updates
        if("meas" in networkName):
            updateMeasNetworkName(node, nodeName, intfName)
        else:
            print(f"Configuring network {networkName} with IPv4 Network {networkAddress}")
            # If this is an edge network and it is a networking node, not the client node, give it a high-range address.
            if(isEdgeNetwork and not nodeDict[nodeName]["isClient"]):
                currentIPv4Address = hostIPList.pop() # Highest available host address in network
            else:
                currentIPv4Address = hostIPList.pop(0) # Lowest available host address in network

            # Add the address to the node
            intf.ip_addr_add(addr=currentIPv4Address, subnet=currentIPNetwork)
            print(f"\t{nodeName} {intf.get_device_name()} = {currentIPv4Address}")
            
            getNewNetwork = True
        
            ### LOG FILE INFO
            logFile[nodeName]["networks"][networkName]["subnet"] = str(currentIPNetwork)
            logFile[nodeName]["networks"][networkName]["ipv4"] = str(currentIPv4Address)
    
    if(getNewNetwork):
        thirdOctet += 1

Configuring network core-node1-node2 with IPv4 Network 192.168.1.0/24
	node1 eth1 = 192.168.1.1
Configuring network core-node1-node2 with IPv4 Network 192.168.1.0/24
	node2 eth2 = 192.168.1.2
Configuring network core-node1-node3 with IPv4 Network 192.168.2.0/24
	node1 eth2 = 192.168.2.1
Configuring network core-node1-node3 with IPv4 Network 192.168.2.0/24
	node3 eth2 = 192.168.2.2
Configuring network core-node2-node3 with IPv4 Network 192.168.3.0/24
	node3 eth1 = 192.168.3.1
Configuring network core-node2-node3 with IPv4 Network 192.168.3.0/24
	node2 eth3 = 192.168.3.2
Configuring network edge-node2-ipnode-1 with IPv4 Network 192.168.4.0/24
	node2 eth1 = 192.168.4.254
Configuring network edge-node2-ipnode-1 with IPv4 Network 192.168.4.0/24
	ipnode-1 eth1 = 192.168.4.1
Configuring network edge-node3-ipnode-2 with IPv4 Network 192.168.5.0/24
	node3 eth3 = 192.168.5.254
Configuring network edge-node3-ipnode-2 with IPv4 Network 192.168.5.0/24
	ipnode-2 eth1 = 192.168.5.1


In [11]:
from FabUtils import FabOrchestrator

try:
    manager = FabOrchestrator(SLICE_NAME)
    
except Exception as e:
    print(f"Exception: {e}")
cmdOff = "sudo sysctl -w net.ipv4.ip_forward=0"
cmdOn = "sudo sysctl -w net.ipv4.ip_forward=1"
manager.executeCommandsParallel(cmdOff, prefixList="ipnode")
manager.executeCommandsParallel(cmdOn, prefixList="n")

Slice name: ospf_no_meas
Slice and nodes were acquired successfully.
Starting command on node ipnode-1
Command to execute: sudo sysctl -w net.ipv4.ip_forward=0
Starting command on node ipnode-2
Command to execute: sudo sysctl -w net.ipv4.ip_forward=0
Waiting for result from node ipnode-1
stdout: net.ipv4.ip_forward = 0

stderr: 
Waiting for result from node ipnode-2
stdout: net.ipv4.ip_forward = 0

stderr: 
Starting command on node node1
Command to execute: sudo sysctl -w net.ipv4.ip_forward=1
Starting command on node node2
Command to execute: sudo sysctl -w net.ipv4.ip_forward=1
Starting command on node node3
Command to execute: sudo sysctl -w net.ipv4.ip_forward=1
Waiting for result from node node1
stdout: net.ipv4.ip_forward = 1

stderr: 
Waiting for result from node node2
stdout: net.ipv4.ip_forward = 1

stderr: 
Waiting for result from node node3
stdout: net.ipv4.ip_forward = 1

stderr: 


In [7]:
import json
## LOG FILE INFO
with open("topo_log.json", "w") as outfile: 
    json.dump(logFile, outfile)